# Introduction à knn
---
Le but de ce TP est de mieux appréhender l'apprentissage via une méthode type 'plus proches voisins'. 
--- 

## Conseils
- Pour chaque question, essayer d'écrire des fonctions plutôt que des scripts
- Eviter d'avoir deux fonctions qui portent le même nom (redéfinition de la fonction), préférer ajouter un paramètre de choix

In [ ]:
# Placer ici vos imports pour la totalité du TP
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
print("Setup Complete")
import numpy as np

import sklearn.model_selection as skl
import sklearn.metrics as skm
import sklearn.neighbors as skn

import os

In [ ]:
# paramètres globaux



---
# Construction d'un jeu de données simple
---

## Ecrire une fonction ```separation```
- Entrée :  un couple (x, y) de flottants
- Sortie : 'A' si y<f(x)  et 'B' sinon, où $f(x) = \frac{x}{3}*\left(2+\cos\left(\frac{x}{6}\right)\right)$

## Ecrire une fonction ```donneesSimples``` 
- Entrée : 
    - ```N``` un entier 
    - ```f``` une fonction prenant en entrée deux flottants
- Sortie : un dataframe de ```N``` échantillons de 3 attributs :
  - ```abscisse``` : tiré en aléatoire uniforme dans [0 ; 100], flottant
  - ```ordonnee``` : tiré en aléatoire uniforme dans [0 ; 100], flottant
  - ```classe = f(abscisse, ordonnée)```
- vous *devez* utiliser un apply de la fonction séparation
  
## Ecrire une fonction ```representation```
Cette fonction génère une représentation graphique des données dans le plan

## Créer une BD ```donnees``` de 300 données
- par application de ```donneesSimples(300, separation)```


## Représenter la base de donnée graphiquement

In [ ]:
def separation(x, y):
    if y < x/3*(2+np.cos(x/6)):
        return 'A'
    else:
        return 'B'
    
def donneesSimples(N, f):
    def fct(data):
        return f(data.Abscisse, data.Ordonnee)
    
    absc = np.random.rand(N)*100
    ord = np.random.rand(N)*100
    
    data = pd.DataFrame({'Abscisse' : absc, 'Ordonnee' : ord})
    data2 = data.apply(fct, 1)
    
    data3 = pd.DataFrame({'Abscisse' : data.Abscisse, 'Ordonnee' : data.Ordonnee, 'Classe' : data2})
    
    return data3

def representation(data):
    sns.scatterplot('Abscisse', 'Ordonnee', 'Classe', data = data)
    
    plot2 = np.linspace(0, 100, 300)
    sns.lineplot(plot2, plot2/3*(2+np.cos(plot2/6)))
    plt.figure()
    

donnees = donneesSimples(300, separation)
                 
display(donnees.head())
representation(donnees)

## En utilisant scikit-learn, séparer ```donnees``` en deux ensembles
Ecrire la fonction , fonction ```creerBases``` qui sépare la base de données en :
- apprentissage (et validation) : 80%
- test 20%  
 
**NB** : l'ensemble d'apprentissage servira également d'ensemble de validation puisqu'on utilisera une validation croisée

In [ ]:
def creerBases(data):
    data_X = data.loc[:, ['Abscisse', 'Ordonnee']]
    data_Y = data.loc[:, 'Classe']
    train_X, test_X, train_Y, test_Y = skl.train_test_split(data_X, data_Y, test_size = 0.2, stratify=donnees.Classe)
    return train_X, test_X, train_Y, test_Y

donnees_train_X, donnees_test_X, donnees_train_Y, donnees_test_Y = creerBases(donnees)

# Théorie et sa mise en pratique dans sklearn
- Rappeler le fonctionnement de la classification par knn
- Expliquer les variations proposées par 'algorithm'
- Expliquer l'influence de 'p' dans la distance de Minkowski (exemples à l'appui)
- Expliquer l'influence de 'weight' (exemples à l'appui)

 Le principe de la classification par plus proche voisin est d'observer les k plus proches voisins (en terme de similitude des attributs) et de prendre la classe la plus représentées.

    BallTree, KDTree, Brute, auto KNeighborsClassifier



# Représenter la surface de séparation en fonction du nombre de voisins
## Fonction de coloration
Ecrire une fonction ```colorationPlan``` qui produit une représentation [0 ; 100]x[0 ; 100] colorée en fonction de la classe attribuée par votre classifieur.
en fonction de la classe attribuée par votre classifieur à chacun de ses points.
- Entrées : 
    - ```classifieur``` une fonction de classification
    - ```NbPts``` nombre de points à prendre dans chaque direction pour discrétiser [0 ; 100]x[0 ; 100]
- Affichage : la coloration du carré demandée en utilisant par exemple un ```scatterplot```

## Observer et analyser l'influence du choix de 'k' sur la classification obtenue
Pour différentes valeurs de ```k``` dans le classifieur, utiliser la fonction ```colorationPlan``` en superposant sur le graphique la ligne de séparation théorique (l'équation du début) en utilisant par exemple un ```lineplot```


In [ ]:
def colorationPlan(cla, NbPts):
    data = pd.DataFrame({'Abscisse' : [], 'Ordonnee' : [], 'Classe' : []})
    temp = 0
    plot = np.linspace(0, 100, NbPts)
    for i in plot:
        for j in plot:
            data.loc[temp] = [i, j, cla.predict([[i, j]])[0]]
            temp+=1
    representation(data)
    
for i in range(1, 11, 1):
    neigh = skn.KNeighborsClassifier(n_neighbors=i)
    neigh.fit(donnees_train_X, donnees_train_Y)

    colorationPlan(neigh, 50)

## Commenter
- Essayer d'expliquer les résultats obtenus
- Entre deux résultats équivalents pour deux valeurs de 'k' différentes, lequel choisir ?

Explications : 

On a des résultats assez différent d'une valeur de k à une autre. Plus notre k sera grand, moins on aura de points orange car il y a une plus grande proportion de bleu que d'orange. Il faut donc réussir à trouver la valeur de k la plus élevée possible en évitant d'avoir un surplus de bleu lié à leur forte quantité.


La valeur de k la plus interessante semble être k = 4 car on vois sur le graphique que c'est le seul a respecter la partie gauche du graphique et donc à ne pas se faire trop influencer par le bleu au dessus de la courbe.

## Recherche des paramètres optimaux
- utiliser une gridsearch pour déterminer les meilleurs paramètres, expliquer la valeur de cv
- valider votre apprentissage sur l'ensemble de validation
- ne pas oublier de comparer la qualité obtenue sur l'ensemble de test et sur l'ensemble de validation

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_params = {
    'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10]
}

grid_search = GridSearchCV(skn.KNeighborsClassifier(), grid_params, verbose = 1, cv = 3, n_jobs=-1)
result = grid_search.fit(donnees.iloc[:,:-1], donnees.iloc[:,-1])

display(result.best_params_)

cv = validation en croix

# Passez temporairement le nombre d'échantillons de la base de données de départ à 100 puis à 700
- Observer l'influence de la quantité de données sur les valeurs optimales des paramètres, expliquer
- Observer l'influence de la quantité de données sur la qualité du résultat obtenu, expliquer
- Observer l'influence de la quantité de données sur le temps de calcul, expliquer

In [ ]:
donnees = donneesSimples(100, separation)
donnees_train_X, donnees_test_X, donnees_train_Y, donnees_test_Y = creerBases(donnees)

grid_search = GridSearchCV(skn.KNeighborsClassifier(), grid_params, verbose = 1, cv = 3, n_jobs=-1)
result = grid_search.fit(donnees.iloc[:,:-1], donnees.iloc[:,-1])

display(result.best_params_)

neigh = skn.KNeighborsClassifier(n_neighbors=3)
neigh.fit(donnees_train_X, donnees_train_Y)

colorationPlan(neigh, 100)

In [ ]:
donnees = donneesSimples(700, separation)
donnees_train_X, donnees_test_X, donnees_train_Y, donnees_test_Y = creerBases(donnees)

grid_search = GridSearchCV(skn.KNeighborsClassifier(), grid_params, verbose = 1, cv = 3, n_jobs=-1)
result = grid_search.fit(donnees.iloc[:,:-1], donnees.iloc[:,-1])

display(result.best_params_)

neigh = skn.KNeighborsClassifier(n_neighbors=3)
neigh.fit(donnees_train_X, donnees_train_Y)

colorationPlan(neigh, 200)

......

Valeur optimale :  
On peut dire que le nombre de voisin optimal n'a rien a voir avec la taille des données.

Résultat :  
Le résultat est beaucoup plus précis en ajoutant plus de points même si j'ai l'impression que cela se sent vraiment quand on a des écart allant dans les centaines de points d'écart.

Temps de calcul :  
Le temps de calcul est plus long (voir même beaucoup plus long). Rien qu'avec le gridSearch on passe de moins d'une seconde à quelque secondes entre 100 et 700 données. Générer le graphique est cependant beaucoup plus long (d'pù le fait que je l'ai généré avec 200 points et non 700).


---
# Effet du bruit
Dans cette partie on va examiner la résistance au bruit de l'algorithme, et son influence sur le choix du nombre de voisins.
---

## Création d'un bruit
Etudier et expliquer le code suivant :
```{.python}
def genere_separation_bruit(mu, sigma):
  def separation_bruit(x,y ):
    classe = 'A' if y<f(x) else 'B'
    if abs(y-f(x)) <norm.ppf(np.random.rand(),mu, sigma):
        classe = 'A' if classe == 'B' else 'B'
    return classe
  return lambda x,y: separation_bruit(x,y)
```

In [ ]:
def genere_separation_bruit(mu, sigma):
    def separation_bruit(x,y ):
        classe = 'A' if y<f(x) else 'B'
        if abs(y-f(x)) <norm.ppf(np.random.rand(),mu, sigma):
            classe = 'A' if classe == 'B' else 'B'
        return classe
    return lambda x,y: separation_bruit(x,y)


f = genere_separation_bruit(10, 1)

donnees = donneesSimples(300, f)

donnees_train_X, donnees_test_X, donnees_train_Y, donnees_test_Y = creerBases(donnees)

neigh = skn.KNeighborsClassifier(n_neighbors=3)
neigh.fit(donnees_train_X, donnees_train_Y)

colorationPlan(neigh, 50)


....   
Ce code génère une fonction de séparation afin de créer un échantillon de données comportant du "bruit" en ajoutant donc des données à notre échantillon.

## Génération des données
Utiliser le code précédent pour générer 500 données bruitées, et analyser à l'aide de graphiques l'effet du paramètre ```et```

## Analyser l'effet de la force du bruit sur les meilleurs paramètres du modèle

## Conclure, et vérifier l'importance des autres paramètres

---
# Influence des corrélations entre les attributs
---
    On travaille à nouveau sur une base de onnées non bruitée. Ajouter 5 colonnes définies par :
- U1 :   $+30 \times X + 0.1 \times Y$
- U2 :   $-400 \times X - 0.01 \times Y$
- U3 :   $-70 \times X + 0.12 \times Y$
- U4 :   $-28 \times X + 1.5 \times Y$
- U3 :   $+510 \times X - 1.2 \times Y$  

Remettre la colonne ```classe``` en dernière colonne.

# Observer la diférence de qualité entre :
- Apprendre en utilisant toutes les observations (x, y, u1, u2, u3, u4, u5)
- Apprendre en n'utilisant que les observations (x,y)


## Expliquer la raison de la différence observée

## Utiliser un preprocessing adapté sur les données afin que la qualité obtenue en apprenant sur le jeu de données (x, y, u1, u2, u3, u4, u5) soit comparable à celle obtenue en n'utilisant que (x, y)... expliquer


---
# Merci d'avoir suivi ce TP, j'espère qu'il vous a aidé à mieux appréhender l'utilisation de knn, et vous a permis de faire quelques pas dans le domaine *passionnant* de l'apprentissage artificiel